<a href="https://colab.research.google.com/github/yveness7/Dating_profiles_project/blob/main/ok_cupid_profiles_7/12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import random

from matplotlib import pyplot as plt
import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download('omw-1.4')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans

import string
import re

import gensim
from gensim import corpora
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, preprocess_string, stem_text
from gensim.models import LdaMulticore, Word2Vec
from gensim.models.coherencemodel import CoherenceModel

from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.manifold import TSNE

import spacy

!pip install git+https://github.com/LIAAD/yake
import yake

!pip install pyldavis
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-7jixmbtd
  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake /tmp/pip-req-build-7jixmbtd
  Resolved https://github.com/LIAAD/yake to commit 374fc1c1c19eb080d5b6115cbb8d4a4324392e54
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.1 MB/s eta 0:00:00
  Created wheel for yake: filename=yake-0.4.8-py2.py3-none-any.whl size=62570 sha256=1e2fe92992674c4dda887de2f6bda938a85fd1a717a4847e4b14c5142cfa7493
  Stored in directory: /tmp/pip-ephem-wheel-cache-d942fodo/wheels/10/9d/33/6a3358fd876c3d7c6c5c139d1496eb4b1618c7d0e15c375584
Successfully built yake
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 37

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np


df = pd.read_csv('/content/drive/MyDrive/clean_ok_cupid.csv')
df


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,age,status,sex,orientation,body_type,diet,drinks,drugs,education,...,essay8,essay9,body_type_,education_,drinks_,drugs_,smokes_,likes_dogs_,likes_cats_,membership
0,0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,curvy,Graduate,socially,never,sometimes,Yes,Yes,8
1,1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,...,i am very open and will share just about anyth...,NaN,average,Graduate,often,sometimes,no,Yes,Yes,10
2,2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...",thin,Graduate,socially,never,no,No,Yes,1
3,3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,...,NaN,you feel so inclined.,thin,Graduate,socially,never,no,No,Yes,1
4,4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,...,NaN,NaN,average,Graduate,socially,never,no,Yes,Yes,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,59941,59,single,f,straight,NaN,NaN,socially,never,graduated from college/university,...,i have a dream to sing at the alconquin in nyc...,you are seeking a long term connection of shar...,average,Graduate,socially,never,no,Yes,No,4
59942,59942,24,single,m,straight,fit,mostly anything,often,sometimes,working on college/university,...,when i was 18 i got a tattoo of waldo somewher...,meh if you made it this far you might as well.,average,Graduate,often,sometimes,no,Yes,Yes,10
59943,59943,42,single,m,straight,average,mostly anything,not at all,never,graduated from masters program,...,please let me think about this more.,we have similar interests.,average,Graduate,not at all,never,no,No,No,2
59944,59944,27,single,m,straight,athletic,mostly anything,socially,often,working on college/university,...,i like walking around in other people's house ...,you are interested and interesting...,average,Graduate,socially,often,sometimes,Yes,Yes,10


In [ ]:
essays = df[['essay0', 'essay1', 'essay2',
               'essay3', 'essay4', 'essay5', 'essay6',
               'essay7', 'essay8', 'essay9']]
essays.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN


In [ ]:
essays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   essay0  54458 non-null  object
 1   essay1  52374 non-null  object
 2   essay2  50308 non-null  object
 3   essay3  48470 non-null  object
 4   essay4  49409 non-null  object
 5   essay5  49096 non-null  object
 6   essay6  46175 non-null  object
 7   essay7  47495 non-null  object
 8   essay8  40721 non-null  object
 9   essay9  47343 non-null  object
dtypes: object(10)
memory usage: 4.6+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
essays = essays.fillna('.') #fill empty entries with '.'
essays.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   essay0  59946 non-null  object
 1   essay1  59946 non-null  object
 2   essay2  59946 non-null  object
 3   essay3  59946 non-null  object
 4   essay4  59946 non-null  object
 5   essay5  59946 non-null  object
 6   essay6  59946 non-null  object
 7   essay7  59946 non-null  object
 8   essay8  59946 non-null  object
 9   essay9  59946 non-null  object
dtypes: object(10)
memory usage: 4.6+ MB


In [ ]:
essays.essay9[100]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"you're a hip girl that likes to explore new places and share familiar ones. you genuinely like people and are optimistic about the fate of the world. you believe in evolution and that humans are innately good creatures, despite what that doucebag just said. you are an independent thinker and a savvy traveler. you like a guy who has confidence, decent looks, and a questioning (not questionable) personality."

In [ ]:
#aggregating user-generated essays
texts = []
for i in range(59946):
    texts.append(''.join(essays.loc[i]))
essays['texts'] = texts
sample = essays['texts'].sample(n=59946, replace=False)
sample = sample.reset_index()
sample['texts'][0:10]



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    i am adventurous. i like to plan trips and pla...
1    i've been described by my closest friends as o...
2    "come swish around, my pretty punk, and keep m...
3    i'm naturally shy when it comes to meeting peo...
4    just your indian guy next door looking for lov...
5    guess i will put on the serious hat and sell s...
6    iam chinese asian boy. i lived in usa about 5 ...
7    well well well...i'm a half-french/half-spanis...
8    hey. i'm from la, lived in both europe and asi...
9    i thoroughly enjoy life, approaching everythin...
Name: texts, dtype: object

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#text preprocessing
nlp = spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words
all_stopwords.add("love")

sample['texts']
sample['texts'] = sample['texts'].apply(lambda x: remove_stopwords(x))
sample_tokens = sample['texts'].apply(lambda x: word_tokenize(x))

sample_clean = sample_tokens.apply(lambda text: " ".join(i for i in text if i not in all_stopwords))
sample_np = sample_clean.apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))
sample_np = sample_np.apply(lambda x: x.lower().strip())
sample_np = sample_np.apply(lambda x: remove_stopwords(x)) #second pass for certainty

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#to gauge text similarity both lemmatisation and stemming were used, with stemming giving more consistent results

sample_stemmed = sample_np.apply(lambda x: stem_text(x))
sample['stems'] = sample_stemmed
sample['stems'][0:10]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    adventur like plan trip plai basketbal foosbal...
1    describ closest friend genuin kind heart peopl...
2    come swish pretti punk danc stai sober man dri...
3    natur shy come meet peopl introduct tend open ...
4    indian gui door look friend intellig complex h...
5    guess hat sell real estat moment pictur driven...
6    iam chines asian boi live usa 5 year like plai...
7    halffrenchhalfspanish profession appar time ga...
8    hei la live europ asia happili live cali enjoi...
9    thoroughli enjoi life approach hope posit atti...
Name: stems, dtype: object

In [ ]:
#Keyword extraction

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#Extracted keyword strings
yaxtract_mod = yake.KeywordExtractor(lan='en', n=1, dedupLim=0.7, top=5, dedupFunc='seqm', windowsSize=1)

keywords = sample['stems'].apply(lambda x: yaxtract_mod.extract_keywords(x))
keywords_clean = keywords.apply(lambda i: list(dict(i).keys()))

keywords_str = keywords_clean.apply(lambda x: ' '.join(x))

#keywords_str.shape()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
keywords_str[0:10]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0        feel enjoi plan windsurf pari
1     kind favorit iphon number friend
2         danc time friend spend thing
3        kiss comic movi vampir friend
4          high energi indian work dai
5            danc food work home music
6          iam usa year softwar chines
7            kind activ long time citi
8            good live pretti bit movi
9    busi pleasur life friend children
Name: stems, dtype: object

In [ ]:
keywords_str.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(59946,)

In [ ]:
keywords_str.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(59946,)

In [ ]:
#Tf-idf vectorization and KMeans

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#vectorization
tf_idf = TfidfVectorizer(lowercase=False)
X = tf_idf.fit_transform(keywords_str)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#selecting the K-value
inertia_vals = []
k_vals = [10, 100, 1000,2000]

for k in k_vals:
  kmns = KMeans(n_clusters=k)
  kmns.fit(X)

  inertia_vals.append(kmns.inertia_)
plt.plot(k_vals, inertia_vals, 'ro-')
plt.xlabel('k-values')
plt.ylabel('inertias')
plt.show()

In [68]:
#We want to add the keywords as a column in the essay df
essays['Keywords'] = keywords_str

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
#We want to add the keywords as a column in the essay df
essays['Keywords'] = keywords_str

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
essays

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,texts,Keywords
0,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,about me: i would love to think that i was so...,feel enjoi plan windsurf pari
1,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,.,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,.,.,i am very open and will share just about anyth...,.,i am a chef: this is what that means. 1. i am ...,kind favorit iphon number friend
2,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,.,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...","i'm not ashamed of much, but writing public te...",danc time friend spend thing
3,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",.,cats and german philosophy,.,.,you feel so inclined.,i work in a library and go to school. . .readi...,kiss comic movi vampir friend
4,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",.,.,.,.,.,hey how's it going? currently vague on the pro...,high energi indian work dai
...,...,...,...,...,...,...,...,...,...,...,...,...
59941,"vibrant, expressive, caring optimist. i love b...",the happiest times have been when life came to...,i make an outstanding osso bucco. i am also ve...,"i am told that people notice my smile, eyes an...",i am an avid movie watcher and follow the broa...,"my family, my dog, italy, words and music!",writing my book.,"running with my dog, finishing up the work wee...",i have a dream to sing at the alconquin in nyc...,you are seeking a long term connection of shar...,"vibrant, expressive, caring optimist. i love b...",peopl food danc music friend
59942,i'm nick. i never know what to write about mys...,currently finishing school for film production...,"filmmaking, photography, graphic design, web d...","dude, i don't know.","movies: hook (the greatest adventure ever!), g...",iphone contact lenses headphones camera tv rem...,i do most of my thinking on the bus to/from wo...,"bringin' home bacon, or drinking and shakin'!",when i was 18 i got a tattoo of waldo somewher...,meh if you made it this far you might as well.,i'm nick. i never know what to write about mys...,extra point zombi pretti kind
59943,"hello! i enjoy traveling, watching movies, and...","i'm a civil engineer, who enjoys helping the c...",- looking at things objectively - getting thin...,i'm quiet until i get used to the environment ...,"last book: ""game change"". movies: bourne serie...",- iphone - friends and family - internet - bay...,"aside from work, how to improve my home.",out enjoying friendly conversation over dinner.,please let me think about this more.,we have similar interests.,"hello! i enjoy traveling, watching movies, and...",friend big movi messag recent
59944,"""all i have in this world are my balls and

In [70]:
essays.to_csv('essays_keywords.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [71]:
sample.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,texts,stems
0,18122,adventurous. like plan trips play basketball f...,adventur like plan trip plai basketbal foosbal...
1,57574,i've described closest friends genuine kind he...,describ closest friend genuin kind heart peopl...
2,57452,"""come swish around, pretty punk, dancing stay ...",come swish pretti punk danc stai sober man dri...
3,23611,"i'm naturally shy comes meeting people, introd...",natur shy come meet peopl introduct tend open ...
4,8209,indian guy door looking love...- really! frien...,indian gui door look friend intellig complex h...


In [74]:
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,age,status,sex,orientation,body_type,diet,drinks,drugs,education,...,essay8,essay9,body_type_,education_,drinks_,drugs_,smokes_,likes_dogs_,likes_cats_,membership
0,0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,curvy,Graduate,socially,never,sometimes,Yes,Yes,8
1,1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,...,i am very open and will share just about anyth...,NaN,average,Graduate,often,sometimes,no,Yes,Yes,10
2,2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...",thin,Graduate,socially,never,no,No,Yes,1
3,3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,...,NaN,you feel so inclined.,thin,Graduate,socially,never,no,No,Yes,1
4,4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,...,NaN,NaN,average,Graduate,socially,never,no,Yes,Yes,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,59941,59,single,f,straight,NaN,NaN,socially,never,graduated from college/university,...,i have a dream to sing at the alconquin in nyc...,you are seeking a long term connection of shar...,average,Graduate,socially,never,no,Yes,No,4
59942,59942,24,single,m,straight,fit,mostly anything,often,sometimes,working on college/university,...,when i was 18 i got a tattoo of waldo somewher...,meh if you made it this far you might as well.,average,Graduate,often,sometimes,no,Yes,Yes,10
59943,59943,42,single,m,straight,average,mostly anything,not at all,never,graduated from masters program,...,please let me think about this more.,we have similar interests.,average,Graduate,not at all,never,no,No,No,2
59944,59944,27,single,m,straight,athletic,mostly anything,socially,often,working on college/university,...,i like walking around in other people's house ...,you are interested and interesting...,average,Graduate,socially,often,sometimes,Yes,Yes,10


In [76]:
essays.loc[0:0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,texts,Keywords
0,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,about me: i would love to think that i was so...,feel enjoi plan windsurf pari


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
